In [ ]:
# %apt-get update
# %apt install -y chromium-chromedriver
%pip install selenium


In [ ]:
import time
import pandas as pd
import json

from datetime import datetime, timedelta
import pytz

import traceback

import os
import os.path
import io

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaIoBaseDownload, MediaFileUpload


from pathlib import Path

# CONFIG

In [ ]:

import pytz
from datetime import datetime

# Define Jakarta timezone
jakarta_tz = pytz.timezone("Asia/Jakarta")

# Get the current datetime in Jakarta timezone
dt = datetime.now(pytz.utc).astimezone(jakarta_tz)

# Extract day, month, and year
day = dt.day
month = dt.month
year = dt.year

### ---------------- CONFIGURATION ----------------------------###
MOVIE_TITLE = 'xxxxxx' # Change the Movie Title as Study Case
MOVIE_XXI_CODE = 'xxxxxx'
SHEET_NAME = 'xxxxxx'  # Check Sheet Name Before Run
FOLDER_DATA_ID = 'xxxxxx'
SHEET_ID = 'xxxxxx'
ats_date = xx
ats_month = xx

PHONE_AUTH = 'xxxxxx' # Add your phone number that has Account on XXI
PASSWORD_AUTH = 'xxxxxx'    # 6 DIGIT PIN

# Define the time range for filtering
start_time = datetime.strptime("14:00", "%H:%M").time()
end_time = datetime.strptime("16:00", "%H:%M").time()
### -----------------------------------------------------------###

file_schedule_today = f'{MOVIE_TITLE}_xxi_schedule_{day}_{month}_{year}.csv'
output_path = f"admission_{MOVIE_TITLE}_{start_time.strftime('%H')}-{end_time.strftime('%H')}_{day}_{month}.csv"

# Print the values (optional)
print(f"Day: {day}, Month: {month}, Year: {year}")
print(file_schedule_today)



# Login

In [ ]:
# Function to type text with a delay between each character
def type_with_delay(element, text, delay=0.2):
    for char in text:
        element.send_keys(char)
        time.sleep(delay)  # Delay between each character

# Setting up Chromium environment
os.environ['PATH'] += ':/usr/lib/chromium-browser/'

# Setup WebDriver with Selenium Wire
options = webdriver.ChromeOptions()
options.add_argument("--disable-blink-features=AutomationControlled")  # Reduce detection
options.add_argument("--headless=new")
options.add_argument('--no-sandbox')  # Bypass OS security model
# options.add_argument('--disable-dev-shm-usage')  # Overcome limited resource problems
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")

driver = webdriver.Chrome(options=options)
# driver.maximize_window()
driver.set_window_size(1920, 1080)
driver.save_screenshot('windows_maximized.png')  # Save screenshot for debugging

driver.get("https://m.21cineplex.com/id/auth/login")
driver.save_screenshot('open web cineplex.png')  # Save screenshot for debugging

# Login
try:
    # Wait for the phone input field to be visible
    phone_input = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "/html/body/section/div/div/div/div[2]/form/div[1]/div[1]/input"))
    )
    type_with_delay(phone_input, PHONE_AUTH)

    # Wait for the password input field to be visible
    password_input = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "/html/body/section/div/div/div/div[2]/form/div[2]/div/input"))
    )
    type_with_delay(password_input, PASSWORD_AUTH)

    # Click the login button
    login_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "/html/body/section/div/div/div/div[2]/form/button"))
    )
    login_button.click()
    time.sleep(3)

    driver.save_screenshot('click login.png')
    # Ask the user to input the OTP manually
    otp_code = input("Enter the 6-digit OTP: ")

    if len(otp_code) != 6 or not otp_code.isdigit():
        print("Invalid OTP format. Please enter exactly 6 digits.")
        driver.quit()
        exit()

    # Wait for OTP input fields to be visible
    otp_inputs = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.XPATH, "/html/body/section/div/div/div/div[2]/div[2]/div/input"))
    )

    # Fill each OTP input box with corresponding digits
    for i in range(6):
        otp_inputs[i].send_keys(otp_code[i])
        time.sleep(0.2)  # Mimic human input delay
    start_login_time = time.time()
    number_of_scrape_attempts = 0

    print("OTP entered successfully!")

except Exception as e:
    driver.save_screenshot('login_error.png')  # Save screenshot for debugging
    driver.quit()
    print("Error:", e)

In [ ]:
try:
    # Wait until the URL changes from the login page (max wait time: 10 seconds)
    WebDriverWait(driver, 20).until(lambda d: d.current_url != 'https://m.21cineplex.com/id/auth/otp?type=Login')

    driver.save_screenshot('after_login.png')

    # Verify login status
    if driver.current_url == 'https://m.21cineplex.com/id/auth/login':
        print('Login Failed')
    elif driver.current_url == 'https://m.21cineplex.com/auth/otp?type=Login':
        print('OTP Failed')
    elif driver.current_url == 'https://m.21cineplex.com/id':
        print('Login Success')
    else:
        print(f'Unexpected URL: {driver.current_url}')

except TimeoutException:
    print("Timeout: Login process took too long.")
    driver.save_screenshot('login_timeout.png')  # Save screenshot for debugging

# Fetch Schedule

## Download Schedule

In [ ]:
def download_schedule():
    """Checks for a specific file in a folder and downloads it if it exists. If not, downloads an alternative file."""
    creds = None
    SCOPES = ["https://www.googleapis.com/auth/drive"]

    # json_data = 
    json_path = "token.json"
    with open(json_path, "w") as json_file:
        json.dump(json_data, json_file)

    if os.path.exists("token.json"):
        creds = Credentials.from_authorized_user_file("token.json", SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file("cred.json", SCOPES)
            creds = flow.run_local_server(port=0)
        with open("token.json", "w") as token:
            token.write(creds.to_json())

    try:
        service = build("drive", "v3", credentials=creds)

        # Folder ID and primary file name
        folder_id = FOLDER_DATA_ID
        primary_file_name = file_schedule_today

        # Function to download a file
        def download_file(file_id, file_name):
            request = service.files().get_media(fileId=file_id)
            file_path = os.path.join(os.getcwd(), file_name)

            with io.FileIO(file_path, "wb") as file:
                downloader = MediaIoBaseDownload(file, request)
                done = False
                while not done:
                    status, done = downloader.next_chunk()
                    print(f"Download {int(status.progress() * 100)}%.")

            print(f"File downloaded successfully: {file_path}")

        def upload_file(file_path, folder_id):
            file_metadata = {
                'name': os.path.basename(file_path),
                'parents': [folder_id]
            }
            media = MediaFileUpload(file_path, resumable=True)
            file = service.files().create(body=file_metadata, media_body=media, fields='id').execute()
            print(f"File uploaded successfully: {file_path} (ID: {file['id']}")

        # Query to check if the primary file exists in the folder
        query = f"'{folder_id}' in parents and name='{primary_file_name}' and trashed=false"
        results = service.files().list(q=query, fields="files(id, name)").execute()
        items = results.get("files", [])

        user_info = service.about().get(fields="user").execute()
        print(f"Authenticated as: {user_info['user']['emailAddress']}")

        if items:
            for item in items:
                print(f"Found file: {item['name']} (ID: {item['id']})")
                download_file(item["id"], item["name"])


    except HttpError as error:
        print(f"An error occurred: {error}")


if __name__ == "__main__":
    download_schedule()


## Upload Schedule

In [ ]:
# Function to upload a file
from googleapiclient.http import MediaIoBaseDownload, MediaFileUpload

creds = None

SCOPES = ["https://www.googleapis.com/auth/drive"]

# json_data = 
# Convert the embedded JSON to a temporary credentials file
json_path = "token.json"
with open(json_path, "w") as json_file:
    json.dump(json_data, json_file)
# Check if token.json exists for credentials
if os.path.exists("token.json"):
    creds = Credentials.from_authorized_user_file("token.json", SCOPES)
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file("cred.json", SCOPES)
        creds = flow.run_local_server(port=0)
    with open("token.json", "w") as token:
        token.write(creds.to_json())

service = build("drive", "v3", credentials=creds)

def upload_file(file_path, folder_id):
    # Check if a file named "cookies.json" already exists in the folder
    query = f"'{folder_id}' in parents and name='{file_path}' and trashed=false"
    results = service.files().list(q=query, fields="files(id, name)").execute()
    items = results.get("files", [])

    # If there are existing files, delete them
    for item in items:
        print(f"Deleting old file: {item['name']} (ID: {item['id']})")
        service.files().delete(fileId=item["id"]).execute()

    file_metadata = {
        'name': os.path.basename(file_path),
        'parents': [folder_id]
    }
    media = MediaFileUpload(file_path, resumable=True)
    file = service.files().create(body=file_metadata, media_body=media, fields='id').execute()
    print(f"File uploaded successfully: {file_path} (ID: {file['id']}")

## Scrape Schedule

In [ ]:
import time
import pytz
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

# List of cities to process

cities = ['AMBON', 'BALIKPAPAN', 'BANDUNG', 'BANJARMASIN', 'BATAM', 'BEKASI', 'BENGKULU', 'BOGOR', 'BONDOWOSO',
          'BONTANG', 'CIANJUR', 'CIKARANG', 'CILEGON', 'CIREBON', 'DENPASAR', 'DEPOK', 'DUMAI', 'GARUT', 'GORONTALO',
          'GRESIK', 'INDRAMAYU', 'JAKARTA', 'JAMBI', 'JAYAPURA', 'JEMBER', 'KARAWANG', 'KEDIRI', 'KENDARI', 'KISARAN',
          'KLATEN', 'KUALA KAPUAS', 'KUPANG', 'LAMPUNG', 'MADIUN', 'MAKASSAR', 'MALANG', 'MAMUJU', 'MANADO', 'MANOKWARI',
          'MATARAM', 'MEDAN', 'PADANG', 'PALANGKARAYA', 'PALEMBANG', 'PALU', 'PANGKAL PINANG', 'PEKALONGAN', 'PEKANBARU',
          'PONTIANAK', 'RANTAU PRAPAT', 'ROKAN HILIR', 'SAMARINDA', 'SEMARANG', 'SIDOARJO', 'SINGKAWANG', 'SOLO', 'SORONG',
          'SUMEDANG', 'SURABAYA', 'TANGERANG', 'TANJUNG PINANG', 'TARAKAN', 'TASIKMALAYA', 'TERNATE', 'TIMIKA', 'YOGYAKARTA']

# Define cinema time zone lists
wit_cinemas = [
    "DIANA MALL XXI", "SORONG XXI", "MANOKWARI CITY MALL XXI", "JAYAPURA XXI", "JATILAND XXI", "AMBON CITY CENTER XXI"
]

wita_cinemas = [
    "PENTACITY XXI", "STUDIO XXI BALIKPAPAN", "STUDIO XXI MAKASSAR", "TSM XXI MAKASSAR", "CITIMALL BONTANG XXI",
    "MALEO TOWN SQUARE XXI", "THE PARK KENDARI XXI", "MANTOS 3 XXI", "MEGA MALL XXI", "TSM XXI BALI", "LEVEL 21 XXI",
    "LEM XXI", "GRAND TARAKAN MALL XXI", "SCP XXI", "E-WALK XXI", "GRAND MALL PALU XXI", "GALERIA XXI", "M`TOS XXI",
    "STUDIO XXI BANJARMASIN", "NIPAH XXI", "SAMARINDA SQUARE XXI","PANAKKUKANG XXI", "LIVING WORLD DENPASAR XXI",
    "GORONTALO XXI", "TRANSMART KUPANG XXI", "AYANI XXI", "CITY CENTRUM XXI", "ICON BALI XXI", "MANTOS 1 XXI",
    "cinema PARK 23 XXI CITY", "MANTOS 1 XXI CITY", "BIG MALL XXI", "ICON BALI IMAX", "PARK 23 XXI CITY",
    "PANAKKUKANG 21 CITY", "PARK 23 XXI", "BEACH WALK XXI", "PANAKKUKANG 21", "GRAND TARAKAN MALL XXI CITY"
]

def click_siap_button(driver):
    try:
        # Wait for the modal to appear
        wait = WebDriverWait(driver, 10)
        siap_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Siap')]")))

        # Click the button
        siap_button.click()
        print("Clicked 'Siap' button successfully.")
        time.sleep(2)  # Small delay after clicking

    except Exception as e:
        print("No 'Siap' button found or failed to click:", e)

# Initialize variables
schedule_data = []
driver.get(f"https://m.21cineplex.com/id/movies/{MOVIE_XXI_CODE}")
time.sleep(3)

# Check for "Film belum tayang" modal
try:
    modal = WebDriverWait(driver, 3).until(
        EC.visibility_of_element_located((By.XPATH, "/html/body/div[2]/div[2]/div/div/div/div/div"))
    )
    print("Modal detected: Film belum tayang.")
    close_button = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.CLASS_NAME, "Button_fill__tHryY"))
    )
    time.sleep(2)
    close_button.click()
    print("Modal closed.")
except Exception:
    print("No modal detected.")

# click_siap_button(driver) # Exclusive for 21+ movies

# Get movie title
movie_title = WebDriverWait(driver, 3).until(
    EC.visibility_of_element_located((By.XPATH, "/html/body/section/div/div[2]/header/div[2]/h1"))
).text

# Get current Jakarta time
jakarta_tz = pytz.timezone("Asia/Jakarta")
current_datetime = datetime.now(jakarta_tz)
current_day, current_month, current_year = current_datetime.day, current_datetime.month, current_datetime.year

def click_jadwal_tab():
    """Ensure the 'Jadwal' tab is clicked and loaded."""
    try:
        jadwal_tab = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.XPATH, "//button[contains(., 'Jadwal')]"))
        )
        driver.execute_script("arguments[0].click();", jadwal_tab)
        print("Clicked on 'Jadwal' tab.")
        time.sleep(2)
    except Exception as e:
        print("Error clicking 'Jadwal' tab:", e)

def convert_to_wib(cinema_name, showtime):
    """Convert showtime to WIB based on cinema location."""
    try:
        showtime_dt = datetime.strptime(showtime, "%H:%M")
        if cinema_name in wit_cinemas:
            return (showtime_dt - timedelta(hours=2)).strftime("%H:%M")  # Convert WIT -> WIB (UTC+9 to UTC+7)
        elif cinema_name in wita_cinemas:
            return (showtime_dt - timedelta(hours=1)).strftime("%H:%M")  # Convert WITA -> WIB (UTC+8 to UTC+7)
        return showtime  # Already in WIB
    except Exception as e:
        print(f"Error converting showtime for {cinema_name}: {e}")
        return showtime

def get_selected_date(driver, date_format="MM/DD/YYYY"):
    """Extracts the currently selected date and formats it correctly."""
    try:
        selected_date_button = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.XPATH, "//button[contains(@class, 'date-active')]"))
        )
        selected_date = selected_date_button.text.strip()
    except Exception:
        driver.refresh()
        print("No date is selected.")
        return None

    # Get current date in Jakarta time
    jakarta_tz = pytz.timezone("Asia/Jakarta")
    current_datetime = datetime.now(jakarta_tz)
    current_day = current_datetime.day
    current_month = current_datetime.month
    current_year = current_datetime.year

    # Extract the selected day
    try:
        selected_day = int(selected_date.split("\n")[1])  # Adjust this based on actual format
    except (IndexError, ValueError):
        print("Failed to extract the selected date.")
        return None

    # Determine the correct month and year
    if current_day <= selected_day:
        schedule_month = current_month
        schedule_year = current_year
    else:
        schedule_month = current_month + 1
        if schedule_month > 12:
            schedule_month = 1
            schedule_year = current_year + 1  # Handle year change
        else:
            schedule_year = current_year

    # Create datetime object
    selected_datetime = datetime(schedule_year, schedule_month, selected_day, tzinfo=jakarta_tz)

    # Return formatted date
    if date_format == "MM/DD/YYYY":
        return selected_datetime.strftime("%m/%d/%Y")
    return selected_datetime.strftime("%d/%m/%Y")


def change_to_target_date(driver):
    """Ensure the selected date is 30/04/2025. as the date of release"""
    global schedule_date  # Ensure schedule_date is defined globally
    schedule_date = get_selected_date(driver, "MM/DD/YYYY")
    # print(f"Selected Date: {schedule_date}")

    # -------------------------------- THIS CODE PART ONLY USED FOR ADVANCED TICKET SALES ---------------------------------------- #

    if schedule_date:
        compare_date = datetime(2025, ats_month, ats_date).strftime("%m/%d/%Y")
        # print(f"Compared Date: {schedule_date} < {compare_date}: {(schedule_date < compare_date)} ")
        if schedule_date < compare_date:
            print(f"Current selected date: {schedule_date}. Changing to {str(ats_date)}/{str(ats_month)}/2025...")
            try:
                button_ats_date = driver.find_element(By.XPATH, f"//button[.//time[text()='{str(ats_date)}']]")
                driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", button_ats_date)
                time.sleep(1)
                driver.execute_script("arguments[0].click();", button_ats_date)
                print(f"Successfully changed to {str(ats_date)}/{str(ats_month)}/2025.")
                schedule_date = datetime(2025, ats_month, ats_date).strftime("%m/%d/%Y")  # Update after change

            except Exception as e:
                print("Error: Unable to change the date.")
                print(e)
    # ---------------------------------------------------------------------------------------------------------------------------- #

def get_current_location():
    """Get the currently selected city from the website."""
    try:
        location_button = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.XPATH, "/html/body/nav/div[2]/div/div[1]/span/div/button/p"))
        )
        return location_button.text.strip()
    except Exception:
        return None


def change_location(city):
    """Change the location to the specified city."""
    while get_current_location() != city:
        print(f"Current location: {get_current_location()}, changing to {city}...")
        try:
            location_button = WebDriverWait(driver, 15).until(
                EC.element_to_be_clickable((By.XPATH, "/html/body/nav/div[2]/div/div[1]/span/div/button"))
            )
            location_button.click()
            city_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, f"//button[span[text()='{city}']]"))
            )
            city_button.click()
            time.sleep(2)
            print(f"Location successfully changed to {city}.")
        except Exception as e:
            driver.save_screenshot(f'Error changing location to {city}.png')  # Save screenshot for debugging
            print(f"Error changing location to {city}: {e}")


def process_cinema_schedules():
    """Extract schedule data for the selected city."""
    try:
        cinema_cards = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, "CardTheaterList_card__58w9m"))
        )

        for cinema_card in cinema_cards:
            cinema_name = cinema_card.find_element(By.TAG_NAME, "h3").text.strip()
            print(f"Processing: {cinema_name}")

            theaters = cinema_card.find_elements(By.CLASS_NAME, "CardTheaterList_card__studio__G_MUn")

            for theater in theaters:
                try:
                    schedule_html = theater.get_attribute("innerHTML")
                    soup = BeautifulSoup(schedule_html, "html.parser")
                    header = soup.select_one(".CardTheaterList_card__studio__header___SXLU")

                    ticket_price = header.find_all("span")[1].text.strip() if header else ""

                    schedule_times = [showtime.text.strip() for showtime in soup.select(".CardTheaterTime_card__header__6Guzt span")]
                    for showtime in schedule_times:
                        schedule_data.append({
                            "movie_title": movie_title,
                            "city": city,
                            "cinema_name": cinema_name,
                            "ticket_price": ticket_price,
                            "schedule_date": schedule_date,  # This is now always defined
                            "showtimes": showtime,
                            "showtimes_WIB": convert_to_wib(cinema_name, showtime)
                        })

                except Exception as e:
                    print(f"Error processing theater for {cinema_name}: {e}")

    except Exception as e:
        print(f"Error processing cinemas: {e}")

file_path = Path(file_schedule_today)  # Adjust the file path if necessary

if file_path.exists():
    print(f"{file_path} exists. Skip Scrape Schedule...")
else:
    print(f"{file_path} doesnt exists. Scraping Schedule...")
    # Iterate through all cities
    for i, city in enumerate(cities):
        df = pd.DataFrame()

        print(f"\n{10*'-'} Processing {city} ({i+1}/{len(cities)}) {10*'-'}")

        # Change to the target city
        change_location(city)
        print(f"Final location: {get_current_location()}")

        # Click the 'Jadwal' tab
        click_jadwal_tab()

        # Change to the correct date if necessary
        change_to_target_date(driver)

        # Check for "Film belum tayang" modal
        try:
            modal = WebDriverWait(driver, 3).until(
                EC.visibility_of_element_located((By.XPATH, "/html/body/div[2]/div[2]/div/div/div/div/div"))
            )
            print("Modal detected: Film belum tayang.")
            close_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "Button_fill__tHryY"))
            )
            time.sleep(2)
            close_button.click()
            print("Modal closed.")
            continue
        except Exception:
            print("No modal detected.")

        # Check if no schedule exists
        try:
            no_schedule_element = WebDriverWait(driver, 5).until(
                EC.visibility_of_element_located((By.XPATH, "//div[@data-testid='not-found-container']"))
            )
            print(f"No schedule available for {city}.")
            continue
        except Exception:
            print(f"Schedule exists for {city}.")

        # Process cinema schedules
        process_cinema_schedules()

    # Final schedule data
    print(schedule_data)


In [ ]:
if len(schedule_data) != 0:
  df = pd.DataFrame(schedule_data)
  df.to_csv(file_schedule_today)
  upload_file(file_schedule_today, FOLDER_DATA_ID)
  df.info()
else:
  df = pd.read_csv(file_schedule_today)
  df.info()

In [ ]:
duplicates = df[df.duplicated(subset=['movie_title', 'cinema_name', 'showtimes'], keep=False)]
display(duplicates.head(20))

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np


hourly_distribution = pd.to_datetime(df['showtimes_WIB'], format='%H:%M').dt.hour

# Define bin range (09:00 to 23:00)
min_hour = 9
max_hour = 23
bins = list(range(min_hour, max_hour + 2))  # +2 because bins are right-exclusive

# Plot using Seaborn
plt.figure(figsize=(10, 5))
sns.set_style("whitegrid")
ax = sns.histplot(hourly_distribution, bins=bins, color="#3498db", edgecolor="black", discrete=True)

# Annotate count labels
counts, _ = np.histogram(hourly_distribution, bins=bins)
for i, count in enumerate(counts):
    bar_x = bins[i]
    ax.text(bar_x, count + 0.05, str(count), ha='center', va='bottom', fontsize=10)

# Final touches
ax.set_xticks(range(min_hour, max_hour + 1))
ax.set_xlabel('Hour of Day (WIB)')
ax.set_ylabel('Number of Showtimes')
ax.set_title('Showtimes Binned by Hour (Starting from 09:00)')
plt.tight_layout()
plt.show()


# Fetch Seat Page

## Filter Show Period

In [ ]:
temp_df = pd.read_csv(file_schedule_today, index_col=0)

# Convert showtimes_WIB to datetime for filtering
temp_df["showtimes_WIB"] = pd.to_datetime(temp_df["showtimes_WIB"], format="%H:%M").dt.time

# Filter the DataFrame
temp_df = temp_df[(temp_df["showtimes_WIB"] >= start_time) & (temp_df["showtimes_WIB"] < end_time)]
temp_df.sort_values(by=["showtimes_WIB", "city"], ascending=[True, True], inplace=True)
# temp_df.sort_values(by="showtimes_WIB", inplace=True, ascending=True)
temp_df.reset_index(drop=True, inplace=True)

print(f"total data on between {start_time.strftime('%H')} - {end_time.strftime('%H')}: {len(temp_df)}")

# Get current Jakarta time + 5 minutes
jakarta_tz = pytz.timezone('Asia/Jakarta')
now_jakarta = datetime.now(jakarta_tz)
cutoff_time = (now_jakarta + timedelta(minutes=5)).time()

# Filter out shows that are already too close or past
temp_df_eta_only = temp_df[temp_df["showtimes_WIB"] > cutoff_time]

# Sort and reset index
temp_df_eta_only = temp_df_eta_only.sort_values(by="showtimes_WIB", ascending=True).reset_index(drop=True)

# Estimate end time
seconds_per_record = 10
# total_records = len(temp_df_eta_only)
total_records = len(temp_df_eta_only)
estimated_end_time = now_jakarta + timedelta(seconds=total_records * seconds_per_record)
eta_minutes = int((estimated_end_time - now_jakarta).total_seconds() // 60)

# Output
print(f"Current Jakarta time: {now_jakarta.strftime('%H:%M:%S')}")
# print(f"Skipping shows that start before: {cutoff_time.strftime('%H:%M')}")
print(f"Remaining records to scrape: {total_records}")
print(f"Estimated end time: {estimated_end_time.strftime('%H:%M:%S')} ({eta_minutes} Minutes)")

# Optional: Show a preview
temp_df.head(10)

In [ ]:
# Convert to datetime.time
temp_df['showtimes_WIB'] = pd.to_datetime(temp_df['showtimes_WIB'], format='%H:%M:%S').dt.time

# Convert to minutes since midnight
temp_df['minutes'] = (
    pd.to_datetime(temp_df['showtimes_WIB'].astype(str), format='%H:%M:%S')
    .dt.hour * 60
    + pd.to_datetime(temp_df['showtimes_WIB'].astype(str), format='%H:%M:%S').dt.minute
)

# Bin into 5-minute intervals
temp_df['time_bin'] = (temp_df['minutes'] // 5) * 5  # floor to nearest 5

# Convert back to HH:MM for readability
temp_df['time_bin'] = pd.to_datetime(temp_df['time_bin'], unit='m').dt.strftime('%H:%M')

# Distribution
distribution = temp_df['time_bin'].value_counts().sort_index()

display(distribution)

temp_df_eta_only.head(10)

In [ ]:
temp_df['filled_seat'] = None
temp_df['total_seat'] = None
temp_df['last_update'] = None
temp_df.info()


## Scrape Seat

In [ ]:
import time
import pytz
import os
from datetime import datetime, timedelta
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import TimeoutException, WebDriverException
from urllib3.exceptions import ReadTimeoutError

# OPTIMIZATION 1: Reduce timeout values
DEFAULT_WAIT_TIME = 10  # Reduced from 20
QUICK_WAIT_TIME = 5     # For less critical operations
MAX_REFRESH_ATTEMPTS = 2  # Limit refresh attempts

start_time_scrape = time.time()
laymer_list = []

# Create error_log_image directory if it doesn't exist
error_log_dir = "error_log_image"
if not os.path.exists(error_log_dir):
    os.makedirs(error_log_dir)
    print(f"Created directory: {error_log_dir}")

# Define Jakarta timezone
jakarta_tz = pytz.timezone("Asia/Jakarta")

def get_jakarta_time():
    return datetime.now(jakarta_tz).strftime("%Y-%m-%d %H:%M:%S")

def safe_refresh_driver(max_attempts=MAX_REFRESH_ATTEMPTS):
    """Safely refresh the driver with error handling and limited attempts"""
    for attempt in range(max_attempts):
        try:
            print(f"Refresh attempt {attempt + 1}/{max_attempts}")
            # OPTIMIZATION 2: Set shorter timeout for refresh operations
            driver.set_page_load_timeout(30)  # 30 seconds instead of default
            driver.refresh()
            time.sleep(3)  # Reduced wait time
            return True
        except (ReadTimeoutError, TimeoutException, WebDriverException) as e:
            print(f"Refresh attempt {attempt + 1} failed: {str(e)[:50]}...")

            if attempt < max_attempts - 1:  # Not the last attempt
                try:
                    # OPTIMIZATION 3: Quick navigation fallback
                    driver.set_page_load_timeout(20)
                    driver.get(f"https://m.21cineplex.com/id/movies/{MOVIE_XXI_CODE}")
                    time.sleep(5)
                    return True
                except Exception as nav_error:
                    print(f"Navigation fallback failed: {nav_error}")
                    continue
            else:
                print("All refresh attempts failed")
                return False
        except Exception as e:
            print(f"Unexpected error during refresh attempt {attempt + 1}: {e}")
            if attempt == max_attempts - 1:
                return False
    return False

def get_current_location():
    try:
        location_button = WebDriverWait(driver, QUICK_WAIT_TIME).until(
            EC.element_to_be_clickable((By.XPATH, "/html/body/nav/div[2]/div/div[1]/span/div/button/p"))
        )
        return location_button.text.strip()
    except Exception:
        return None

def change_location(city):
    """Change the location to the specified city with timeout optimization."""
    max_location_attempts = 3
    attempt = 0

    while get_current_location() != city and attempt < max_location_attempts:
        attempt += 1
        print(f"Location change attempt {attempt}/{max_location_attempts}: {get_current_location()} → {city}")
        try:
            if attempt > 1:
                location_button = WebDriverWait(driver, DEFAULT_WAIT_TIME + 10).until(
                    EC.element_to_be_clickable((By.XPATH, "/html/body/nav/div[2]/div/div[1]/span/div/button"))
                )
            else:
                location_button = WebDriverWait(driver, DEFAULT_WAIT_TIME).until(
                    EC.element_to_be_clickable((By.XPATH, "/html/body/nav/div[2]/div/div[1]/span/div/button"))
                )
            location_button.click()

            city_button = WebDriverWait(driver, DEFAULT_WAIT_TIME).until(
                EC.element_to_be_clickable((By.XPATH, f"//button[span[text()='{city}']]"))
            )
            city_button.click()
            time.sleep(1)  # Reduced wait time

            if get_current_location() == city:
                print(f"Location successfully changed to {city}.")
                return True
        except Exception as e:
            print(f"Error changing location to {city} (attempt {attempt}): {e}")
            time.sleep(1)

    if attempt >= max_location_attempts:
        print(f"Failed to change location after {max_location_attempts} attempts")
        return False
    return True

def change_to_target_date(driver):
    """Ensure the selected date is set to the ATS release date if required."""
    global schedule_date
    schedule_date = get_selected_date(driver, "MM/DD/YYYY")

    if not schedule_date:
        print("No selected date found.")
        return False

    try:
        target_date = datetime(2025, ats_month, ats_date).strftime("%m/%d/%Y")
        if schedule_date < target_date:
            print(f"Current selected date: {schedule_date}. Attempting to change to {target_date}...")

            try:
                button_ats_date = driver.find_element(
                    By.XPATH, f"//button[not(contains(@class,'disabled')) and .//time[text()='{str(ats_date)}']]"
                )
                driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", button_ats_date)
                time.sleep(0.5)  # Reduced wait time
                driver.execute_script("arguments[0].click();", button_ats_date)

                updated_date = get_selected_date(driver, "MM/DD/YYYY")
                if updated_date == target_date:
                    print(f"Successfully changed to {target_date}.")
                    schedule_date = updated_date
                    return True
                else:
                    print("Failed to confirm the date change.")
                    return False

            except Exception as e:
                print(f"Target date button ({ats_date}/{ats_month}/2025) not found.")
                return False
    except Exception as e:
        print("Error while comparing or changing date:", e)
        return False

    return True

# OPTIMIZATION 4: Add circuit breaker pattern
consecutive_failures = 0
MAX_CONSECUTIVE_FAILURES = 5

def should_skip_due_to_failures():
    global consecutive_failures
    return consecutive_failures >= MAX_CONSECUTIVE_FAILURES

def reset_failure_counter():
    global consecutive_failures
    consecutive_failures = 0

def increment_failure_counter():
    global consecutive_failures
    consecutive_failures += 1

def process_row(index, row):
    global is_first

    # OPTIMIZATION 5: Circuit breaker check
    if should_skip_due_to_failures():
        print(f"Skipping row {index} due to consecutive failures. Manual intervention may be needed.")
        # return False

    try:
        # OPTIMIZATION 6: Set page load timeout before navigation
        driver.set_page_load_timeout(45)
        driver.get(f"https://m.21cineplex.com/id/movies/{MOVIE_XXI_CODE}")

        if is_first:
            try:
                modal = WebDriverWait(driver, 3).until(
                    EC.visibility_of_element_located((By.XPATH, "/html/body/div[2]/div[2]/div/div/div/div/div"))
                )
                print("Modal detected: Film belum tayang.")
                close_button = WebDriverWait(driver, QUICK_WAIT_TIME).until(
                    EC.element_to_be_clickable((By.CLASS_NAME, "Button_fill__tHryY"))
                )
                time.sleep(1)  # Reduced wait time
                close_button.click()
                print("Modal closed.")
                is_first = False
            except Exception:
                print("No modal detected.")
                is_first = False

        movie_title = WebDriverWait(driver, DEFAULT_WAIT_TIME).until(
                EC.visibility_of_element_located((By.XPATH, "/html/body/section/div/div[2]/header/div[2]/h1"))
            )
        movie_title = movie_title.text

        # OPTIMIZATION 7: Optimize location change
        if not change_location(row['city']):
            print(f"Failed to change location to {row['city']}, skipping row {index}")
            increment_failure_counter()
            return False

        target_date_success = change_to_target_date(driver)
        if not target_date_success:
            print("Target date unavailable. Proceeding with currently available date.")

        try:
            selected_date_button = WebDriverWait(driver, DEFAULT_WAIT_TIME).until(
                EC.presence_of_element_located((By.XPATH, "//button[contains(@class, 'date-active')]"))
            )
            selected_date = selected_date_button.text.strip()

            selected_date = get_selected_date(driver)
            if selected_date:
                compare_date = datetime(2025, ats_month, ats_date).strftime("%m/%d/%Y")
                comparison_result = "before" if selected_date <= compare_date else "after or equal"
                print(f"Using date: {selected_date} (comparison: {comparison_result} target date)")
            else:
                print("Warning: Could not determine selected date format, but proceeding with scraping.")

        except Exception:
            if not safe_refresh_driver():
                print("Failed to refresh driver, skipping row")
                increment_failure_counter()
                return False
            print("No date is selected, but continuing...")

        # Wait for cinema list with reduced timeout
        WebDriverWait(driver, DEFAULT_WAIT_TIME).until(
            EC.presence_of_element_located((By.CLASS_NAME, "ScheduleList_theater__list___g0DG"))
        )

        WebDriverWait(driver, DEFAULT_WAIT_TIME).until(
            EC.presence_of_element_located((By.CLASS_NAME, "CardTheaterList_card__58w9m"))
        )

        # Find cinemas
        cinemas = driver.find_elements(By.CLASS_NAME, "CardTheaterList_card__58w9m")

        # Find target cinema
        cinema_found = False
        for cinema in cinemas:
            try:
                cinema_name = cinema.find_element(By.TAG_NAME, "h3").text.strip()
                if cinema_name == row['cinema_name']:
                    cinema_found = True
                    print(f"Cinema found: {cinema_name}")
                    try:
                        WebDriverWait(driver, DEFAULT_WAIT_TIME).until(
                            EC.element_to_be_clickable((By.CLASS_NAME, "CardTheaterList_collapse__hfYeY"))
                        )
                        toggle_button = cinema.find_element(By.CLASS_NAME, "CardTheaterList_collapse__hfYeY")
                        driver.execute_script("arguments[0].scrollIntoView();", toggle_button)
                        driver.execute_script("arguments[0].click();", toggle_button)
                    except Exception:
                        print("Card already opened")

                    # Wait for showtimes
                    WebDriverWait(cinema, DEFAULT_WAIT_TIME).until(
                        EC.visibility_of_element_located((By.CLASS_NAME, "CardTheaterTime_card__DHZ4u"))
                    )
                    showtimes = cinema.find_elements(By.CLASS_NAME, "CardTheaterTime_card__DHZ4u")

                    showtime_found = False
                    for showtime in showtimes:
                        try:
                            if "CardTheaterTime_disabled__di2PI" in showtime.get_attribute("class"):
                                continue

                            time_text = showtime.find_element(By.CLASS_NAME, "CardTheaterTime_card__header__6Guzt").text.strip()
                            if time_text == row['showtimes']:
                                showtime_found = True
                                ActionChains(driver).move_to_element(showtime).click().perform()

                                try:
                                    modal = WebDriverWait(driver, QUICK_WAIT_TIME).until(
                                        EC.presence_of_element_located((By.CLASS_NAME, "Modal_modal__card__R6ylt"))
                                    )
                                    continue_buttons = modal.find_elements(By.CLASS_NAME, "Button_fill__tHryY")
                                    if continue_buttons:
                                        continue_button = continue_buttons[-1]
                                        driver.execute_script("arguments[0].scrollIntoView(true);", continue_button)
                                        WebDriverWait(driver, DEFAULT_WAIT_TIME).until(EC.element_to_be_clickable(continue_button)).click()
                                    break
                                except Exception:
                                    print(f"Modal did not appear after clicking showtime {time_text}")
                                    continue
                        except Exception as e:
                            print(f"Error processing showtime: {e}")

                    if not showtime_found:
                        print(f"Showtime {row['showtimes']} not found or available")
                    break
            except Exception as e:
                print(f"Error processing cinema: {e}")

        if not cinema_found:
            print(f"Cinema {row['cinema_name']} not found")
            increment_failure_counter()
            safe_refresh_driver()
            return False

        # Count seats
        try:
            seat_container = WebDriverWait(driver, DEFAULT_WAIT_TIME).until(
                EC.presence_of_element_located((By.CLASS_NAME, "TheaterSeats_seats__list__C3ldb"))
            )

            WebDriverWait(seat_container, DEFAULT_WAIT_TIME).until(
                EC.any_of(
                    EC.presence_of_element_located((By.CLASS_NAME, "TheaterSeats_free__NVolQ")),
                    EC.presence_of_element_located((By.CLASS_NAME, "TheaterSeats_sold__D3BVv"))
                )
            )

            free_seats = seat_container.find_elements(By.CLASS_NAME, "TheaterSeats_free__NVolQ")
            booked_seats = seat_container.find_elements(By.CLASS_NAME, "TheaterSeats_sold__D3BVv")
            total_seats = len(free_seats) + len(booked_seats)

            temp_df.at[index, 'filled_seat'] = len(booked_seats)
            temp_df.at[index, 'total_seat'] = total_seats
            temp_df.at[index, 'last_update'] = get_jakarta_time()

            if (len(booked_seats)/total_seats) >= 0.5:
                print(f"Filled seats: {len(booked_seats)}/{total_seats}")
                laymer_list.append(f"{temp_df.at[index, 'cinema_name']} ({temp_df.at[index, 'city']}) on {temp_df.at[index, 'showtimes']} with {len(booked_seats)} filled seats")

            reset_failure_counter()  # Reset on success
            return True

        except Exception as e:
            print(f"Error counting seats: {e}")
            temp_df.at[index, 'filled_seat'] = None
            temp_df.at[index, 'total_seat'] = None
            temp_df.at[index, 'last_update'] = get_jakarta_time()
            increment_failure_counter()
            return False

    except Exception as e:
        print(f"Unexpected error for row {index}: {e}")
        increment_failure_counter()
        return False

# OPTIMIZATION 8: Improved retry logic with backoff
is_first = True
retry_count = 0
MAX_RETRIES = 2  # Reduced from implicit unlimited retries

while temp_df['filled_seat'].isnull().any() and retry_count <= MAX_RETRIES:
    print(f"Retrying rows with null values... (retry count: {retry_count})")

    null_rows = temp_df[temp_df['filled_seat'].isnull()]
    if len(null_rows) == 0:
        break

    for index, row in null_rows.iterrows():
        jakarta_tz = pytz.timezone('Asia/Jakarta')
        now_jakarta = datetime.now(jakarta_tz)
        adjusted_time = now_jakarta + timedelta(minutes=5)
        adjusted_time_only = adjusted_time.time()

        showtime = row['showtimes_WIB']

        base_string = f"Processing {row['movie_title']} on cinema {row['cinema_name']} CITY: {row['city']} at {row['showtimes']} ({len(temp_df[temp_df['filled_seat'].notna()])}/{len(temp_df)})"
        total_length = 120
        num_dashes = max(0, total_length - len(base_string))

        print(f"{'-' * (num_dashes // 2)} {base_string} {'-' * (num_dashes // 2)}")

        # Skip rows where the showtime is in the past
        if (showtime < adjusted_time_only) or (showtime < now_jakarta.time()):
            print(f"Showtime already closed: {row['showtimes']}")
            print(f"Current time in WIB : {now_jakarta.time()}")
            continue

        success = process_row(index, row)

        if not success:
            try:
                screenshot_path = os.path.join(error_log_dir, f"Error on {index}.png")
                driver.save_screenshot(screenshot_path)
                print(f"Error screenshot saved: {screenshot_path}")
            except Exception as screenshot_error:
                print(f"Error saving screenshot for row {index}: {screenshot_error}")

            print(f"Row {index} failed, will retry in next iteration if retries remain")

            # Handle special cases
            current_url = driver.current_url
            if "login" in current_url:
                print("Detected login page, stopping execution")
                print(f"Current url: {driver.current_url}")
                logged_out_time = time.time()
                login_last = start_login_time - logged_out_time
                print(f"Login session lasted: {login_last} seconds after {number_of_scrape_attempts} scrape attempts")

                break
            elif "myorder" in current_url:
                print("Detected 'My Order' page, saving HTML and navigating back")
                try:
                    # Save the page source to the same error_log_dir
                    html_path = os.path.join(error_log_dir, f"MyOrder_on_{index}.html")
                    with open(html_path, "w", encoding="utf-8") as f:
                        f.write(driver.page_source)
                    print(f"MyOrder HTML saved: {html_path}")

                    # Navigate back to movie page
                    driver.get(f"https://m.21cineplex.com/id/movies/{MOVIE_XXI_CODE}")
                    time.sleep(3)
                except Exception as nav_error:
                    print(f"Navigation error: {nav_error}")
                    break

            # OPTIMIZATION 9: Exponential backoff on errors
            backoff_time = min(2 ** retry_count, 10)  # Max 10 seconds
            print(f"Waiting {backoff_time} seconds before continuing...")
            time.sleep(backoff_time)

        if success:
            number_of_scrape_attempts += 1

    retry_count += 1

# Final summary
remaining_nulls = len(temp_df[temp_df['filled_seat'].isnull()])
if remaining_nulls > 0:
    print(f"WARNING: {remaining_nulls} rows still have null values after all retries")

end_time_scrape = time.time()
elapsed_time = end_time_scrape - start_time_scrape
print(f"Total execution time: {elapsed_time:.2f} seconds")

# Upload Data

In [ ]:
laymer_list

In [ ]:
temp_df['filled_seat'].sum()

In [ ]:
temp_df.head()

In [ ]:
df_na = temp_df[temp_df['filled_seat'].isna()]
temp_df = temp_df[temp_df['filled_seat'].notna()]
temp_df.head(10)

In [ ]:
temp_df.to_csv(output_path, index=False)

In [ ]:
from datetime import datetime
import json
import os
import pandas as pd
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# sheet_json_data = 
# Convert the embedded JSON to a temporary credentials file
json_path = "token_sheet.json"
with open(json_path, "w") as json_file:
    json.dump(sheet_json_data, json_file)

SCOPES = ["https://www.googleapis.com/auth/spreadsheets"]
SPREADSHEET_ID = SHEET_ID
TARGET_SHEET_NAME = SHEET_NAME
CITY_DATA_SHEET_NAME = "city_data"

MATCH_COLS = ['movie_title', 'cinema_name', 'showtimes', 'schedule_date']

def authenticate_google_sheets():
    """Authenticate and return Google Sheets API service"""
    creds = None
    if os.path.exists("token_sheet.json"):
        creds = Credentials.from_authorized_user_file("token_sheet.json", SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file("credentials.json", SCOPES)
            creds = flow.run_local_server(port=0)
        with open("token_sheet.json", "w") as token:
            token.write(creds.to_json())

    return build("sheets", "v4", credentials=creds)

def fetch_city_data(service):
    """Fetch city data from the 'city_data' sheet"""
    try:
        result = service.spreadsheets().values().get(
            spreadsheetId=SPREADSHEET_ID,
            range=f"{CITY_DATA_SHEET_NAME}!A:B"
        ).execute()
        city_data = result.get("values", [])

        # Convert city data into a dictionary with cinema name as the key and daerah (city) as the value
        city_dict = {}
        for row in city_data[1:]:  # Skip header row
            cinema_name, daerah = row
            city_dict[cinema_name] = daerah

        return city_dict
    except HttpError as err:
        print(f"Error fetching city data: {err}")
        return {}

def fetch_existing_data(service):
    """Fetch existing data from the target sheet and return as DataFrame."""
    try:
        result = service.spreadsheets().values().get(
            spreadsheetId=SPREADSHEET_ID,
            range=f"{TARGET_SHEET_NAME}!A:K"
        ).execute()
        rows = result.get("values", [])
        if not rows or len(rows) < 2:
            return pd.DataFrame(columns=[
                'movie_title', 'cinema_name', 'exhibitor', 'city', 'showtimes',
                'total_seat', 'filled_seat', 'occupancy_rate',
                'schedule_datetime', 'last_update', 'no_show'
            ])
        return pd.DataFrame(rows[1:], columns=[
            'movie_title', 'cinema_name', 'exhibitor', 'city', 'showtimes',
            'total_seat', 'filled_seat', 'occupancy_rate',
            'schedule_datetime', 'last_update', 'no_show'
        ])
    except HttpError as err:
        print(f"Error fetching existing sheet data: {err}")
        return pd.DataFrame()

def get_no_show(show_hm):
    """Determine the show number based on the 'Show_HM' time."""
    try:
        show_time = datetime.strptime(show_hm, "%H:%M").time()

        if datetime.strptime("09:00", "%H:%M").time() <= show_time < datetime.strptime("13:00", "%H:%M").time():
            return "Show 1"
        elif datetime.strptime("13:00", "%H:%M").time() <= show_time < datetime.strptime("15:00", "%H:%M").time():
            return "Show 2"
        elif datetime.strptime("15:00", "%H:%M").time() <= show_time < datetime.strptime("17:00", "%H:%M").time():
            return "Show 3"
        elif datetime.strptime("17:00", "%H:%M").time() <= show_time < datetime.strptime("20:00", "%H:%M").time():
            return "Show 4"
        elif datetime.strptime("20:00", "%H:%M").time() <= show_time <= datetime.strptime("23:59", "%H:%M").time():
            return "Show 5"
        else:
            return "Other"
    except ValueError:
        return "Invalid Time"


def process_sheet_data(service, new_df, city_dict):
    existing_df = fetch_existing_data(service)
    display(existing_df)
    updates = []
    new_rows = []

    for idx, row in new_df.iterrows():
        match_condition = (
            (existing_df['movie_title'] == row['movie_title']) &
            (existing_df['cinema_name'] == row['cinema_name']) &
            (existing_df['showtimes'] == row['showtimes']) &
            (pd.to_datetime(existing_df['schedule_datetime'], format='%d/%m/%Y %H:%M:%S', errors='coerce')
              .dt.strftime('%m/%d/%Y') == row['schedule_date'])
        )
        matched = existing_df[match_condition]

        filled_seat = row["filled_seat"]
        total_seat = row["total_seat"]
        last_update = row["last_update"].strftime('%Y-%m-%d %H:%M:%S') if isinstance(row["last_update"], pd.Timestamp) else row["last_update"]
        occupancy_rate = filled_seat / total_seat if total_seat > 0 else 0
        show_date_time = row['schedule_date'] + ' ' + row['showtimes'] + ':00'
        no_show = get_no_show(row["showtimes"])
        city = city_dict.get(row["cinema_name"], "")
        exhibitor = "XXI"

        if not matched.empty:
                sheet_row_index = matched.index[0] + 2  # +2 for header + 1-indexing
                existing_last_update_str = matched.iloc[0]["last_update"]
                try:
                    existing_last_update = pd.to_datetime(existing_last_update_str, format='%d/%m/%Y %H:%M:%S', errors='coerce')
                    new_last_update = pd.to_datetime(row["last_update"], errors='coerce')
                except Exception as e:
                    print(f"⚠️ Error parsing last_update: {e}")
                    existing_last_update = new_last_update = None

                if new_last_update and existing_last_update and new_last_update > existing_last_update:
                    updates.extend([
                        {"range": f"{TARGET_SHEET_NAME}!F{sheet_row_index}:F{sheet_row_index}", "values": [[total_seat]]},
                        {"range": f"{TARGET_SHEET_NAME}!G{sheet_row_index}:G{sheet_row_index}", "values": [[filled_seat]]},
                        {"range": f"{TARGET_SHEET_NAME}!H{sheet_row_index}:H{sheet_row_index}", "values": [[occupancy_rate]]},
                        {"range": f"{TARGET_SHEET_NAME}!J{sheet_row_index}:J{sheet_row_index}", "values": [[last_update]]},
                    ])
                    # print(f"🔁 Updated row {sheet_row_index} — new data is fresher.")
                else:
                    print(f"⏩ Skipped update for row {sheet_row_index} — older or same timestamp.")

        else:
            # ➕ Append new row
            new_rows.append([
                row["movie_title"], row["cinema_name"], exhibitor, city, row["showtimes"],
                total_seat, filled_seat, occupancy_rate,
                show_date_time, last_update, no_show
            ])

    # ✅ Batch update
    if updates:
        service.spreadsheets().values().batchUpdate(
            spreadsheetId=SPREADSHEET_ID,
            body={"valueInputOption": "RAW", "data": updates}
        ).execute()
        print(f"✅ Updated {len(updates)//4} rows")

    # ➕ Append new rows
    if new_rows:
        service.spreadsheets().values().append(
            spreadsheetId=SPREADSHEET_ID,
            range=f"{TARGET_SHEET_NAME}!A:K",
            valueInputOption="RAW",
            insertDataOption="INSERT_ROWS",
            body={"values": new_rows}
        ).execute()
        print(f"➕ Appended {len(new_rows)} new rows")

# Run it all
service = authenticate_google_sheets()
city_dict = fetch_city_data(service)
process_sheet_data(service, temp_df, city_dict)


# Quit